# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Couchard Darius - Parent Paul - Donne Stefano

## Pump it Up: Data Mining the Water Table
####  April 29, 2021


# 2) Data Pre-Processing




In [1]:
require(tidyr)
require(dplyr)
require(plyr)
training_set<-read.csv("../Data/TrainingSet/4910797b-ee55-40a7-8668-10efd5c1b960.csv",header=TRUE) # loads the training set csv file (it's magic)
dim(training_set) # dimension of the set 
names(training_set) # names of the variables

traninin_labels<-read.csv("../Data/TrainingLabel/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv", header=TRUE) # Loads the corresponding labels


Loading required package: tidyr

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘tidyr’”
Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: plyr

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘plyr’”


[1] 59400    40

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group"

## How to enhance the data set :

After having analyzed the data set and assessed each variable relevance, it's needed to standardize the datas.

First, it's needed to remove empty values in the table (Imputation): each NaN or empty cell has to be removed or replaced, different solution exists : 
* Mapping NaN<-0 for nominal categorical variables
* Replace missing value by mean of column for numerical variables

Then, modifications have to be made depending on the nature of the data :
* If a column (variable) consists in continous numerical values : standardization is applied such has obtaining a new column with a mean value of 0 and a standard deviation of 1 (**longitude**)
* If a column is made of ordinal categorical variable (hierarchy between categories) : map each string to a numerical value (**water_quality**)
* In case of nominal categorical variable : apply one hot encoding -> create new column (with binary values) for each category (**source_type**)
<br/>


## Methods are implemented to apply these changes :

In [2]:
# METHOD TO REASSIGN EMPTY VALUES
NaN_handler_categorical <- function(column_name) { # input : column_name (name of the variable)
    training_set[training_set[,column_name] == "",column_name ] <- 0 # select row where column element is empty string "" and assign to 0
    training_set[is.na(training_set[,column_name]),column_name] <- 0 # select row where column element is NaN and assign to 0
    return(training_set)
}


# METHOD TO REASSIGN EMPTY VALUES
NaN_handler_num<- function(column_name) { # input : column_name (name of the variable)
    mean_col <- mean(training_set[,column_name], na.rm = TRUE)
    training_set[is.na(training_set[,column_name]),column_name] <- mean_col
    return(training_set)
}



In [3]:
# STANDARDIZATION METHOD FOR VARIABLES WITH CONTINUOUS NUMERICAL VALUES
Standardization <- function(column_name){ # input : column_name (name of the variable)
    mean_col <- mean(training_set[,column_name], na.rm = TRUE) # mean of the variable
    sd_col <- sd(training_set[,column_name], na.rm = TRUE) # standard deviation of the variable
    training_set[,column_name]<-(training_set[,column_name]-mean_col)/sd_col # apply the transformation
    # now for the whole column : mean = 0 and sd = 1
    return(training_set)
}

In [4]:
# HANDLING OF NOMINAL CATEGORICAL VARIABLES
# before using : change notebook IOPub data rate limit with Jupyter  notebook --NotebookApp.iopub_data_rate_limit=jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
Nom_cat_handle <- function(column_name){
    training_set <- training_set %>% mutate(value = 1)  %>% spread(column_name, value,  fill = 0 )
    return(training_set)
}


## Process the data-set :

In [5]:
# COLUMNS TO DROP 
column_to_drop<-c("wpt_name","amount_tsh","date_recorded","gps_height","num_private","public_meeting","recorded_by",
                 "scheme_name","payment","quantity_group","source_class","subvillage","waterpoint_type",
                 "extraction_type","extraction_type_group","water_quality","source")
#TODO : adds others columns to drop
training_set<-training_set[,!(names(training_set) %in% column_to_drop)] # drop the desired column

In [6]:
# MANAGE SIMPLE NOMINAL CATEGORICAL VARIABLE
#TODO : apply one-hot encoding to other needed variables
training_set<-Nom_cat_handle("basin") # apply one-hot-encoding to the basin related column
training_set<-Nom_cat_handle("payment_type")
training_set<-Nom_cat_handle("quality_group")
training_set<-Nom_cat_handle("quantity")
training_set<-Nom_cat_handle("waterpoint_type_group")
training_set<-Nom_cat_handle("extraction_type_class")
training_set<-Nom_cat_handle("source_type")
# ...

ERROR: Error in spread(., column_name, value, fill = 0): impossible de trouver la fonction "spread"


In [7]:
# MANAGE ORDINAL CATEGORICAL VARIABLE

# variable PERMIT
# remap True : 1 , False : 0, Missing "" : NA
training_set$permit <- mapvalues(training_set$permit, 
          from=c("True","False",""), 
          to=c(1,0,NA))
training_set <- transform(training_set, permit = as.numeric(permit)) # transform column data type (char to int)
# replace missing value by mean of column
training_set<-NaN_handler_num("permit")
# Standardize Variable
training_set<-Standardization("permit")



ERROR: Error in mapvalues(training_set$permit, from = c("True", "False", ""), : impossible de trouver la fonction "mapvalues"


In [8]:
# MANAGE POPULATION VARIABLE
# NaN have to be replaced by mean of region population
# column 1 = region code, column 2 = population mean in this region
region_code_frame <- data.frame("region" = unique(training_set$region_code),"mean_pop" = NA)
for(row in 1: nrow(region_code_frame) ){ 
    sel <-training_set[which(training_set[,"region_code"]==region_code_frame[row,1],),"population"] # select pop row with corresponding region
    region_code_frame[row,"mean_pop"]<-mean(sel[sel!=0],na.rm = TRUE)
}

# TODO : some region population mean are NaN , find a solution (same problem with region and district_code variables)
# temporairement on remplace les NaN par le mean des autres valeurs dans region_code_frame
region_code_frame[which(is.na(region_code_frame[,2])),2]<-mean(region_code_frame[,2],na.rm=TRUE)

# replace NaN value of population by mean region values
index<-which(training_set[,"population"]==0)
for(elem in index){
    training_set[elem,"population"]<-region_code_frame[training_set[elem,"region_code"],2] # replace NaN by their mean region value
}

# Now Standardize Population Variable 
training_set<-Standardization("population")

#TODO: Manage other tricky variables

In [9]:
# Write the pre-processed data into a new XLS file
library("writexl")
write_xlsx(training_set,"../Data/PreProcess/processed_training_data.csv") 

ERROR: Error in library("writexl"): there is no package called ‘writexl’


In [47]:
library(mltools)
library(data.table)

one_hot_encode <- function(data, column, replace_NA_b) {
    # Factor the data
    data[,column] <- factor(data[,column])
    # One hot encode
    hot <- one_hot(data.table(data[,column]))
    # Drop column
    data[,column] <- NULL
    # Places new columns
    data[,names(hot)] <- hot
    return (data)
}

training_set <- one_hot_encode(training_set, "scheme_management")


In [48]:
names(training_set)

[1] "id"                    "funder"                "installer"            
 [4] "longitude"             "latitude"              "basin"                
 [7] "region"                "region_code"           "district_code"        
[10] "lga"                   "ward"                  "population"           
[13] "permit"                "construction_year"     "extraction_type_class"
[16] "management"            "management_group"      "payment_type"         
[19] "quality_group"         "quantity"              "source_type"          
[22] "waterpoint_type_group" "V1_"                   "V1_Company"           
[25] "V1_None"               "V1_Other"              "V1_Parastatal"        
[28] "V1_Private operator"   "V1_SWC"                "V1_Trust"             
[31] "V1_VWC"                "V1_Water authority"    "V1_Water Board"       
[34] "V1_WUA"                "V1_WUG"